In [2]:
import pymongo
import requests
import json
import numpy as np
import pandas as pd
import time
from tqdm import tqdm
import datetime
from credentials import client_id,client_secret
from ipynb.fs.full.repo_collector import cleanLinks, getOrgNRepo, renameCollections, org_name, repo_name

In [3]:
def addMISCDetails(repo_id, org_id, issue_number, list_details):
    updt_list = []    
    add_details = {"org_id":org_id, "repo_id":repo_id, "issue_id":issue_number}
    for ele in list_details:
        ele.update(add_details)
        updt_list.append(ele)            
    return updt_list

In [11]:
def populateToDB(url, org_name, repo_name, issue_number, db_name):
    
    response = requests.get(url,auth=(client_id, client_secret))
    print("GitHub Rest API rate limit remaining: "+str(response.headers.get('X-RateLimit-Remaining')))
       
    if response.json():    
        raw_list = response.json()
    #     print(len(raw_list))

        mongo_client = pymongo.MongoClient("mongodb://127.0.0.1:27017/")
        repodb = mongo_client["repo_collector"]
        repo_coll = repodb[db_name]

    #     print(raw_list)
        repo_id, org_id = getOrgNRepo(org_name,repo_name,"tf_repo")
    #     print(str(repo_id)+" | "+str(org_id))

        updt_list = addMISCDetails(repo_id, org_id, issue_number, raw_list)
        #print(issue_raw_list[-1])

        repo_coll.insert_many(updt_list)
        print("Inserted rows to "+db_name)

        if response.headers.get('Link'):
            print(response.headers.get('Link'))
            link = response.headers.get('Link')
        #     print(link)
            clean_links = cleanLinks(link)        
        #     print(clean_links)

            df = pd.DataFrame(clean_links, columns =['url', 'rel'])
        #     print(df)
            
            if int(response.headers.get('X-RateLimit-Remaining')) <= 2:
                print("Please wait for 30 minutes..............")
                time.sleep(60*30)
            
            while "next" in df['rel'].values:

                index = int(df[df['rel']=='next'].index.values)
    #             print(df['url'][index])
                url = df['url'][index]
                response = requests.get(url,auth=(client_id, client_secret))
                raw_list = response.json()
                print("GitHub Rest API rate limit remaining: "+str(response.headers.get('X-RateLimit-Remaining')))
                
                updt_list = addMISCDetails(repo_id, org_id, issue_number, raw_list)

                repo_coll.insert_many(updt_list)

                link = response.headers.get('Link')
                clean_links = cleanLinks(link)
                df = pd.DataFrame(clean_links, columns =['url', 'rel'])

                if int(response.headers.get('X-RateLimit-Remaining')) <= 2:
                    print("Please wait for 30 minutes..............")
                    time.sleep(60*30)

        mongo_client.close()
    print("Inserting data into "+db_name+" is done.")

In [12]:
mongo_client = pymongo.MongoClient("mongodb://127.0.0.1:27017/")
repodb = mongo_client["repo_collector"]
issues_coll = repodb["repo_issuesV1.1"]

issue_result = list(issues_coll.find({},{"number": 1, "_id":0}))
# print(issue_result[0].get("number"))
mongo_client.close()

In [13]:
if len(issue_result)>0:
    renameCollections("repo_commentsV1.1", "repo_comments")
    renameCollections("repo_eventsV1.1", "repo_events")
    for issue in issue_result:
    #     print(issue.get("number"))
        issue_number = str(issue.get("number"))
        comments_url = "https://api.github.com/repos/"+org_name+"/"+repo_name+"/issues/"+issue_number+"/comments?state=all&per_page=100"
        events_url = "https://api.github.com/repos/"+org_name+"/"+repo_name+"/issues/"+issue_number+"/events?state=all&per_page=100"

        populateToDB(comments_url, org_name, repo_name, issue_number, "repo_commentsV1.1")
        populateToDB(events_url, org_name, repo_name, issue_number, "repo_eventsV1.1")
    print("----------------------- Done inserting events and comments into DB ------------------------------")

GitHub Rest API rate limit remaining: 4596
Inserted rows to repo_commentsV1.1
Inserting data into repo_commentsV1.1 is done.
GitHub Rest API rate limit remaining: 4595
Inserted rows to repo_eventsV1.1
Inserting data into repo_eventsV1.1 is done.
GitHub Rest API rate limit remaining: 4594
Inserted rows to repo_commentsV1.1
Inserting data into repo_commentsV1.1 is done.
GitHub Rest API rate limit remaining: 4593
Inserted rows to repo_eventsV1.1
Inserting data into repo_eventsV1.1 is done.
GitHub Rest API rate limit remaining: 4592
Inserted rows to repo_commentsV1.1
Inserting data into repo_commentsV1.1 is done.
GitHub Rest API rate limit remaining: 4591
Inserted rows to repo_eventsV1.1
Inserting data into repo_eventsV1.1 is done.
GitHub Rest API rate limit remaining: 4590
Inserted rows to repo_commentsV1.1
Inserting data into repo_commentsV1.1 is done.
GitHub Rest API rate limit remaining: 4589
Inserted rows to repo_eventsV1.1
Inserting data into repo_eventsV1.1 is done.
GitHub Rest API 